## 获取数据及数据探索

In [1]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)

from sklearn import *
from sklearn.metrics import mean_squared_error
from sklearn import preprocessing

from datetime import datetime, date, timedelta
import lightgbm as lgb
from xgboost import XGBRegressor

import gc
import os

In [2]:
data = {
    'tra': pd.read_csv('./recruit-restaurant-visitor-forecasting/air_visit_data.csv'),
    'as': pd.read_csv('./recruit-restaurant-visitor-forecasting/air_store_info.csv'),
    'hs': pd.read_csv('./recruit-restaurant-visitor-forecasting/hpg_store_info.csv'),
    'ar': pd.read_csv('./recruit-restaurant-visitor-forecasting/air_reserve.csv'),
    'hr': pd.read_csv('./recruit-restaurant-visitor-forecasting/hpg_reserve.csv'),
    'id': pd.read_csv('./recruit-restaurant-visitor-forecasting/store_id_relation.csv'),
    'tes': pd.read_csv('./recruit-restaurant-visitor-forecasting/sample_submission.csv'),
    'hol': pd.read_csv('./recruit-restaurant-visitor-forecasting/date_info.csv').rename(columns={'calendar_date':'visit_date'})
    }

In [3]:
data["hol"].head()

,visit_date,day_of_week,holiday_flg
0,2016-01-01,Friday,1
1,2016-01-02,Saturday,1
2,2016-01-03,Sunday,1
3,2016-01-04,Monday,0
4,2016-01-05,Tuesday,0


In [4]:
# OneHot
def encode_onehot(df, column_name):
    feature_df = pd.get_dummies(df[column_name], prefix=column_name)
    new_df = pd.concat([df.drop([column_name], axis=1), feature_df], axis=1)
    return new_df

# LabelEncoder
def encode_count(df, column_name):
    lbl = preprocessing.LabelEncoder()
    lbl.fit(list(df[column_name].values))
    df[column_name] = lbl.fit_transform(list(df[column_name].values))
    return df

## 特征工程

In [5]:
data["as"] = encode_count(data["as"], "air_genre_name")
data["as"] = encode_count(data["as"], "air_area_name")

data["hs"] = encode_count(data["hs"], "hpg_genre_name")
data["hs"] = encode_count(data["hs"], "hpg_area_name")

调整hol的特征

In [6]:
data["hol"].columns = ["visit_date","day_of_week","holiday_flg"]
data["hol"]["day_of_week"] = data["hol"]["day_of_week"].replace({"Monday":1,"Tuesday":2,"Wednesday":3,"Thursday":4,"Friday":5,"Saturday":6,"Sunday":7})
data["hol"]["holiday"] = ((data["hol"]["day_of_week"] >= 6) | (data["hol"]["holiday_flg"]==1)).astype(int)
del data["hol"]["day_of_week"]

In [7]:
data["hol"].head()

,visit_date,holiday_flg,holiday
0,2016-01-01,1,1
1,2016-01-02,1,1
2,2016-01-03,1,1
3,2016-01-04,0,0
4,2016-01-05,0,0


调整tra的visitors特征

In [8]:
data["tra"]["visitors"] = data["tra"]["visitors"].apply(lambda x: np.log1p(float(x)) if float(x) > 0 else 0)

调整hr和ar的特征，增加时间差特征

In [9]:
data["hr"] = pd.merge(data["hr"], data["id"], how="inner", on=["hpg_store_id"])

data["ar"]["reserve_date"] = data["ar"]["reserve_datetime"].apply(lambda x:x.split(" ")[0])
data["ar"]["visit_date"] = data["ar"]["visit_datetime"].apply(lambda x:x.split(" ")[0])

data["hr"]["reserve_date"] = data["hr"]["reserve_datetime"].apply(lambda x:x.split(" ")[0])
data["hr"]["visit_date"] = data["hr"]["visit_datetime"].apply(lambda x:x.split(" ")[0])
# 时间差
data["ar"]["reserve_datetime_diff"] = (pd.to_datetime(data["ar"]["visit_date"]) - pd.to_datetime(data["ar"]["reserve_date"])).dt.days
data["hr"]["reserve_datetime_diff"] = (pd.to_datetime(data["hr"]["visit_date"]) - pd.to_datetime(data["hr"]["reserve_date"])).dt.days

拆开tes数据中的id

In [10]:
data["tes"]["air_store_id"] = data["tes"]["id"].apply(lambda x: "_".join(x.split("_")[:2]))
data["tes"]["visit_date"] = data["tes"]["id"].apply(lambda x:x.split("_")[2])
del data["tes"]["visitors"]

In [11]:
data["tra"] = data["tra"].merge(data["id"], on=["air_store_id"], how="left")

In [12]:
data["tes"].head()

,id,air_store_id,visit_date
0,air_00a91d42b08b08d9_2017-04-23,air_00a91d42b08b08d9,2017-04-23
1,air_00a91d42b08b08d9_2017-04-24,air_00a91d42b08b08d9,2017-04-24
2,air_00a91d42b08b08d9_2017-04-25,air_00a91d42b08b08d9,2017-04-25
3,air_00a91d42b08b08d9_2017-04-26,air_00a91d42b08b08d9,2017-04-26
4,air_00a91d42b08b08d9_2017-04-27,air_00a91d42b08b08d9,2017-04-27


In [13]:
data["tra"].head()

,air_store_id,visit_date,visitors,hpg_store_id
0,air_ba937bf13d40fb24,2016-01-13,3.258097,NaN
1,air_ba937bf13d40fb24,2016-01-14,3.496508,NaN
2,air_ba937bf13d40fb24,2016-01-15,3.401197,NaN
3,air_ba937bf13d40fb24,2016-01-16,3.135494,NaN
4,air_ba937bf13d40fb24,2016-01-18,1.945910,NaN


In [14]:
data["ar"].head()

,air_store_id,visit_datetime,reserve_datetime,reserve_visitors,reserve_date,visit_date,reserve_datetime_diff
0,air_877f79706adbfb06,2016-01-01 19:00:00,2016-01-01 16:00:00,1,2016-01-01,2016-01-01,0
1,air_db4b38ebe7a7ceff,2016-01-01 19:00:00,2016-01-01 19:00:00,3,2016-01-01,2016-01-01,0
2,air_db4b38ebe7a7ceff,2016-01-01 19:00:00,2016-01-01 19:00:00,6,2016-01-01,2016-01-01,0
3,air_877f79706adbfb06,2016-01-01 20:00:00,2016-01-01 16:00:00,2,2016-01-01,2016-01-01,0
4,air_db80363d35f10926,2016-01-01 20:00:00,2016-01-01 01:00:00,5,2016-01-01,2016-01-01,0


In [15]:
data["hr"].head()

,hpg_store_id,visit_datetime,reserve_datetime,reserve_visitors,air_store_id,reserve_date,visit_date,reserve_datetime_diff
0,hpg_878cc70b1abc76f7,2016-01-01 19:00:00,2016-01-01 15:00:00,4,air_db80363d35f10926,2016-01-01,2016-01-01,0
1,hpg_878cc70b1abc76f7,2016-01-02 19:00:00,2016-01-02 14:00:00,2,air_db80363d35f10926,2016-01-02,2016-01-02,0
2,hpg_878cc70b1abc76f7,2016-01-03 18:00:00,2016-01-02 20:00:00,6,air_db80363d35f10926,2016-01-02,2016-01-03,1
3,hpg_878cc70b1abc76f7,2016-01-06 20:00:00,2016-01-04 22:00:00,3,air_db80363d35f10926,2016-01-04,2016-01-06,2
4,hpg_878cc70b1abc76f7,2016-01-11 18:00:00,2016-01-11 14:00:00,2,air_db80363d35f10926,2016-01-11,2016-01-11,0


In [16]:
print("tra中访问最早时间:%s" % min(data["tra"]["visit_date"]))
print("tra中访问最晚时间:%s" % max(data["tra"]["visit_date"]))
print("————————分割线————————")
print("hr中预约最早时间:%s" % min(data["hr"]["reserve_date"]))
print("hr中预约最晚时间:%s" % max(data["hr"]["reserve_date"]))
print("hr中访问最早时间:%s" % min(data["hr"]["visit_date"]))
print("hr中访问最晚时间:%s" % max(data["hr"]["visit_date"]))
print("————————分割线————————")
print("ar中预约最早时间:%s" % min(data["ar"]["reserve_date"]))
print("ar中预约最晚时间:%s" % max(data["ar"]["reserve_date"]))
print("ar中访问最早时间:%s" % min(data["ar"]["visit_date"]))
print("ar中访问最晚时间:%s" % max(data["ar"]["visit_date"]))
print("————————分割线————————")

tra中访问最早时间:2016-01-01
tra中访问最晚时间:2017-04-22
————————分割线————————
hr中预约最早时间:2016-01-01
hr中预约最晚时间:2017-04-22
hr中访问最早时间:2016-01-01
hr中访问最晚时间:2017-05-31
————————分割线————————
ar中预约最早时间:2016-01-01
ar中预约最晚时间:2017-04-22
ar中访问最早时间:2016-01-01
ar中访问最晚时间:2017-05-31
————————分割线————————


## 时序特征

### 特征架子(扩展特征)

In [17]:
def feat_sum(df, df_feature, fe,value,name=""):
    #当我们 一个值去匹配另一组个数不一致的时候进行统计特征的时候，往往会使用groupby 。数学公式
    df_count = pd.DataFrame(df_feature.groupby(fe)[value].sum()).reset_index()
    if not name:
        df_count.columns = fe + [value+"_%s_sum" % ("_".join(fe))]
    else:
        df_count.columns = fe + [name]
    df = df.merge(df_count, on=fe, how="left").fillna(0)
    return df


def feat_mean(df, df_feature, fe,value,name=""):
    df_count = pd.DataFrame(df_feature.groupby(fe)[value].mean()).reset_index()
    if not name:
        df_count.columns = fe + [value+"_%s_mean" % ("_".join(fe))]
    else:
        df_count.columns = fe + [name]
    df = df.merge(df_count, on=fe, how="left").fillna(0)
    return df

def feat_count(df, df_feature, fe,value,name=""):
    df_count = pd.DataFrame(df_feature.groupby(fe)[value].count()).reset_index()
    if not name:
        df_count.columns = fe + [value+"_%s_count" % ("_".join(fe))]
    else:
        df_count.columns = fe + [name]
    df = df.merge(df_count, on=fe, how="left").fillna(0)
    return df

import pandas as pd
import numpy as np
from datetime import datetime
from sklearn import preprocessing

def encode_onehot(df,column_name):
    feature_df=pd.get_dummies(df[column_name], prefix=column_name)
    all = pd.concat([df.drop([column_name], axis=1),feature_df], axis=1)
    return all

def encode_count(df,column_name):
    lbl = preprocessing.LabelEncoder()
    lbl.fit(list(df[column_name].values))
    df[column_name] = lbl.transform(list(df[column_name].values))
    return df

def merge_count(df,columns,value,cname):
    add = pd.DataFrame(df.groupby(columns)[value].count()).reset_index()
    add.columns=columns+[cname]
    df=df.merge(add,on=columns,how="left")
    return df

def merge_nunique(df,columns,value,cname):
    add = pd.DataFrame(df.groupby(columns)[value].nunique()).reset_index()
    add.columns=columns+[cname]
    df=df.merge(add,on=columns,how="left")
    return df

def merge_median(df,columns,value,cname):
    add = pd.DataFrame(df.groupby(columns)[value].median()).reset_index()
    add.columns=columns+[cname]
    df=df.merge(add,on=columns,how="left")
    return df

def merge_mean(df,columns,value,cname):
    add = pd.DataFrame(df.groupby(columns)[value].mean()).reset_index()
    add.columns=columns+[cname]
    df=df.merge(add,on=columns,how="left")
    return df

def merge_sum(df,columns,value,cname):
    add = pd.DataFrame(df.groupby(columns)[value].sum()).reset_index()
    add.columns=columns+[cname]
    df=df.merge(add,on=columns,how="left")
    return df

def merge_max(df,columns,value,cname):
    add = pd.DataFrame(df.groupby(columns)[value].max()).reset_index()
    add.columns=columns+[cname]
    df=df.merge(add,on=columns,how="left")
    return df

def merge_min(df,columns,value,cname):
    add = pd.DataFrame(df.groupby(columns)[value].min()).reset_index()
    add.columns=columns+[cname]
    df=df.merge(add,on=columns,how="left")
    return df

def merge_std(df,columns,value,cname):
    add = pd.DataFrame(df.groupby(columns)[value].std()).reset_index()
    add.columns=columns+[cname]
    df=df.merge(add,on=columns,how="left")
    return df
# df['sda']=df.groupby(columns)[value].transform('std')

# df['aggmean']=df.groupby(columns)[value].transform('mean')

def merge_var(df,columns,value,cname):
    add = pd.DataFrame(df.groupby(columns)[value].var()).reset_index()
    add.columns=columns+[cname]
    df=df.merge(add,on=columns,how="left")
    return df

def feat_count(df, df_feature, fe,value,name=""):
    df_count = pd.DataFrame(df_feature.groupby(fe)[value].count()).reset_index()
    if not name:
        df_count.columns = fe + [value+"_%s_count" % ("_".join(fe))]
    else:
        df_count.columns = fe + [name]
    df = df.merge(df_count, on=fe, how="left").fillna(0)
    return df

def feat_nunique(df, df_feature, fe,value,name=""):
    df_count = pd.DataFrame(df_feature.groupby(fe)[value].nunique()).reset_index()
    if not name:
        df_count.columns = fe + [value+"_%s_nunique" % ("_".join(fe))]
    else:
        df_count.columns = fe + [name]
    df = df.merge(df_count, on=fe, how="left").fillna(0)
    return df

def feat_mean(df, df_feature, fe,value,name=""):
    df_count = pd.DataFrame(df_feature.groupby(fe)[value].mean()).reset_index()
    if not name:
        df_count.columns = fe + [value+"_%s_mean" % ("_".join(fe))]
    else:
        df_count.columns = fe + [name]
    df = df.merge(df_count, on=fe, how="left").fillna(0)
    return df

def feat_kernelMedian(df, df_feature, fe, value, pr, name=""):
    def get_median(a, pr=pr):
        a = np.array(a)
        x = a[~np.isnan(a)]
        n = len(x)
        weight = np.repeat(1.0, n)
        idx = np.argsort(x)
        x = x[idx]
        if n<pr.shape[0]:
            pr = pr[n,:n]
        else:
            scale = (n-1)/2.
            xxx = np.arange(-(n+1)/2.+1, (n+1)/2., step=1)/scale
            yyy = 3./4.*(1-xxx**2)
            yyy = yyy/np.sum(yyy)
            pr = (yyy*n+1)/(n+1)
        ans = np.sum(pr*x*weight) / float(np.sum(pr * weight))
        return ans

    df_count = pd.DataFrame(df_feature.groupby(fe)[value].apply(get_median)).reset_index()
    if not name:
        df_count.columns = fe + [value+"_%s_mean" % ("_".join(fe))]
    else:
        df_count.columns = fe + [name]
    df = df.merge(df_count, on=fe, how="left").fillna(0)
    return df

def feat_std(df, df_feature, fe,value,name=""):
    df_count = pd.DataFrame(df_feature.groupby(fe)[value].std()).reset_index()
    if not name:
        df_count.columns = fe + [value+"_%s_std" % ("_".join(fe))]
    else:
        df_count.columns = fe + [name]
    df = df.merge(df_count, on=fe, how="left").fillna(0)
    return df

def feat_median(df, df_feature, fe,value,name=""):
    df_count = pd.DataFrame(df_feature.groupby(fe)[value].median()).reset_index()
    if not name:
        df_count.columns = fe + [value+"_%s_median" % ("_".join(fe))]
    else:
        df_count.columns = fe + [name]
    df = df.merge(df_count, on=fe, how="left").fillna(0)
    return df

def feat_max(df, df_feature, fe,value,name=""):
    df_count = pd.DataFrame(df_feature.groupby(fe)[value].max()).reset_index()
    if not name:
        df_count.columns = fe + [value+"_%s_max" % ("_".join(fe))]
    else:
        df_count.columns = fe + [name]
    df = df.merge(df_count, on=fe, how="left").fillna(0)
    return df

def feat_min(df, df_feature, fe,value,name=""):
    df_count = pd.DataFrame(df_feature.groupby(fe)[value].min()).reset_index()
    if not name:
        df_count.columns = fe + [value+"_%s_min" % ("_".join(fe))]
    else:
        df_count.columns = fe + [name]
    df = df.merge(df_count, on=fe, how="left").fillna(0)
    return df

def feat_sum(df, df_feature, fe,value,name=""):
    df_count = pd.DataFrame(df_feature.groupby(fe)[value].sum()).reset_index()
    if not name:
        df_count.columns = fe + [value+"_%s_sum" % ("_".join(fe))]
    else:
        df_count.columns = fe + [name]
    df = df.merge(df_count, on=fe, how="left").fillna(0)
    return df

def feat_var(df, df_feature, fe,value,name=""):
    df_count = pd.DataFrame(df_feature.groupby(fe)[value].var()).reset_index()
    if not name:
        df_count.columns = fe + [value+"_%s_var" % ("_".join(fe))]
    else:
        df_count.columns = fe + [name]
    df = df.merge(df_count, on=fe, how="left").fillna(0)
    return df

def feat_quantile(df, df_feature, fe,value,n,name=""):
    df_count = pd.DataFrame(df_feature.groupby(fe)[value].quantile(n)).reset_index()
    if not name:
        df_count.columns = fe + [value+"_%s_quantile" % ("_".join(fe))]
    else:
        df_count.columns = fe + [name]
    df = df.merge(df_count, on=fe, how="left").fillna(0)
    return df

### 创建新特征

In [18]:
# 创造新特征以及连接新特征
def create_features(df_label,df_train,df_air_reserve,df_hpg_reserve):
    """
    df_label：0<= tra.day_gap < nday
    df_train：tra.day_gap < 0
    df_air_reserve：ar.day_gap < 0
    df_hpg_reserve：hr.day_gap < 0
    """
    # 增加新特征，与as hol连接
    df_train=date_handle(df_train)
    df_label=date_handle(df_label)
    
    # sum，数值型特征的累加作为新特征
    # df_label和df_air_reserve累加的拼接
    df_label=feat_sum(df_label,df_air_reserve,["air_store_id","visit_date"],"reserve_datetime_diff","air_reserve_datetime_diff_sum")
    df_label=feat_sum(df_label,df_air_reserve,["air_store_id","visit_date"],"reserve_visitors","air_reserve_visitors_sum")
    df_label=feat_sum(df_label,df_air_reserve,["visit_date"],"reserve_visitors","air_date_reserve_visitors_sum")
    # df_label和df_hpg_reserve累加的拼接
    df_label=feat_sum(df_label,df_hpg_reserve,["air_store_id","visit_date"],"reserve_datetime_diff","hpg_reserve_datetime_diff_sum")
    df_label=feat_sum(df_label,df_hpg_reserve,["air_store_id","visit_date"],"reserve_visitors","hpg_reserve_visitors_sum")
    df_label=feat_sum(df_label,df_hpg_reserve,["visit_date"],"reserve_visitors","hpg_date_reserve_visitors_sum")
    # mean，数值型特征的累加作为新特征
    # df_label和df_air_reserve平均值的拼接
    df_label=feat_mean(df_label,df_air_reserve,["air_store_id","visit_date"],"reserve_datetime_diff","air_reserve_datetime_diff_mean")
    df_label=feat_mean(df_label,df_air_reserve,["air_store_id","visit_date"],"reserve_visitors","air_reserve_visitors_mean")
    df_label=feat_mean(df_label,df_air_reserve,["visit_date"],"reserve_visitors","air_date_reserve_visitors_mean")
    # df_label和df_hpg_reserve平均值的拼接
    df_label=feat_mean(df_label,df_hpg_reserve,["air_store_id","visit_date"],"reserve_datetime_diff","hpg_reserve_datetime_diff_mean")
    df_label=feat_mean(df_label,df_hpg_reserve,["air_store_id","visit_date"],"reserve_visitors","hpg_reserve_visitors_mean")
    df_label=feat_mean(df_label,df_hpg_reserve,["visit_date"],"reserve_visitors","hpg_date_reserve_visitors_mean")
    
    
    # 选择day_gap值的范围(air和hpg)
    for i in [7,14,35,63,140]:
        # 筛选出day_gap大于某个数的数据
        df_air_reserve_select=df_air_reserve[df_air_reserve.day_gap>=-i].copy()
        df_hpg_reserve_select=df_hpg_reserve[df_hpg_reserve.day_gap>=-i].copy()
        # 与feat_sum类似
        # air的sum分别和count，mean连接
        date_air_reserve=pd.DataFrame(df_air_reserve_select.groupby(["air_store_id","visit_date"]).reserve_visitors.sum()).reset_index()
        date_air_reserve.columns=["air_store_id","visit_date","reserve_visitors_sum"]
        date_air_reserve=feat_count(date_air_reserve,df_air_reserve_select,["air_store_id","visit_date"],"reserve_visitors","reserve_visitors_count")
        date_air_reserve=feat_mean(date_air_reserve,df_air_reserve_select,["air_store_id","visit_date"],"reserve_visitors","reserve_visitors_mean")
        # hpg的sum分别和count，mean连接
        date_hpg_reserve=pd.DataFrame(df_hpg_reserve_select.groupby(["air_store_id","visit_date"]).reserve_visitors.sum()).reset_index()
        date_hpg_reserve.columns=["air_store_id","visit_date","reserve_visitors_sum"]
        date_hpg_reserve=feat_count(date_hpg_reserve,df_hpg_reserve_select,["air_store_id","visit_date"],"reserve_visitors","reserve_visitors_count")
        date_hpg_reserve=feat_mean(date_hpg_reserve,df_hpg_reserve_select,["air_store_id","visit_date"],"reserve_visitors","reserve_visitors_mean")
        
        # 增加新特征，与as, hol连接
        date_air_reserve=date_handle(date_air_reserve)
        date_hpg_reserve=date_handle(date_hpg_reserve)
        
        # 周末和flg=1满足条件是否
        date_air_reserve["holiday"] = ((date_air_reserve["weekday"]>=5) | (date_air_reserve["holiday_flg"]==1)).astype(int)
        date_hpg_reserve["holiday"] = ((date_hpg_reserve["weekday"]>=5) | (date_hpg_reserve["holiday_flg"]==1)).astype(int)
        
        ####df_label分别和部分的air hpg连接
        # id和weekday关联人数
        df_label=feat_mean(df_label,date_air_reserve,["air_store_id","weekday"],"reserve_visitors_sum", "air_reserve_visitors_sum_weekday_mean_%s"%i)
        df_label=feat_mean(df_label,date_hpg_reserve,["air_store_id","weekday"],"reserve_visitors_sum", "hpg_reserve_visitors_sum_weekday_mean_%s"%i)
        df_label=feat_mean(df_label,date_air_reserve,["air_store_id","weekday"],"reserve_visitors_mean", "air_reserve_visitors_mean_weekday_mean_%s"%i)
        df_label=feat_mean(df_label,date_hpg_reserve,["air_store_id","weekday"],"reserve_visitors_mean", "hpg_reserve_visitors_mean_weekday_mean_%s"%i)
        df_label=feat_mean(df_label,date_air_reserve,["air_store_id","weekday"],"reserve_visitors_count", "air_reserve_visitors_count_weekday_mean_%s"%i)
        df_label=feat_mean(df_label,date_hpg_reserve,["air_store_id","weekday"],"reserve_visitors_count", "hpg_reserve_visitors_count_weekday_mean_%s"%i)
        # id和holiday关联人数
        df_label=feat_mean(df_label,date_air_reserve,["air_store_id","holiday"],"reserve_visitors_sum", "air_reserve_visitors_sum_holiday_mean_%s"%i)
        df_label=feat_mean(df_label,date_hpg_reserve,["air_store_id","holiday"],"reserve_visitors_sum", "hpg_reserve_visitors_sum_holiday_mean_%s"%i)
        df_label=feat_mean(df_label,date_air_reserve,["air_store_id","holiday"],"reserve_visitors_mean", "air_reserve_visitors_mean_holiday_mean_%s"%i)
        df_label=feat_mean(df_label,date_hpg_reserve,["air_store_id","holiday"],"reserve_visitors_mean", "hpg_reserve_visitors_mean_holiday_mean_%s"%i)
        df_label=feat_mean(df_label,date_air_reserve,["air_store_id","holiday"],"reserve_visitors_count", "air_reserve_visitors_count_holiday_mean_%s"%i)
        df_label=feat_mean(df_label,date_hpg_reserve,["air_store_id","holiday"],"reserve_visitors_count", "hpg_reserve_visitors_count_holiday_mean_%s"%i)
    
    ####df_label和总的df_train连接
    # id和day，weekday关联人数
    df_label = feat_mean(df_label, df_train, ["air_store_id","day","weekday"], "visitors", "air_day_mean")
    # id和day，weekday关联人数
    df_label = feat_mean(df_label, df_train, ["air_store_id","day","holiday"], "visitors", "air_holiday_mean")
    
    ####df_label和部分的df_train连接
    # 选择day_gap值的范围(train)
    for i in [21,35,63,140,280,350,420]:
         # 筛选出day_gap大于某个数的数据
        df_select=df_train[df_train.day_gap>=-i].copy()
        
        df_label=feat_median(df_label, df_select, ["air_store_id"], "visitors", "air_median_%s"%i)
        df_label=feat_mean(df_label,df_select,["air_store_id"],"visitors", "air_mean_%s"%i)
        
        df_label=feat_max(df_label,df_select,["air_store_id"],"visitors","air_max_%s"%i)
        df_label=feat_min(df_label,df_select,["air_store_id"],"visitors","air_min_%s"%i)
        df_label=feat_std(df_label,df_select,["air_store_id"],"visitors","air_std_%s"%i)
        df_label=feat_count(df_label,df_select,["air_store_id"],"visitors","air_count_%s"%i)

        
        df_label=feat_mean(df_label,df_select,["air_store_id","weekday"],"visitors", "air_week_mean_%s"%i)
        df_label=feat_max(df_label,df_select,["air_store_id","weekday"],"visitors","air_week_max_%s"%i)
        df_label=feat_min(df_label,df_select,["air_store_id","weekday"],"visitors","air_week_min_%s"%i)
        df_label=feat_std(df_label,df_select,["air_store_id","weekday"],"visitors","air_week_std_%s"%i)
        df_label=feat_count(df_label,df_select,["air_store_id","weekday"],"visitors","air_week_count_%s"%i)

        df_label=feat_mean(df_label,df_select,["air_store_id","holiday"],"visitors", "air_holiday_mean_%s"%i)
        df_label=feat_max(df_label,df_select,["air_store_id","holiday"],"visitors","air_holiday_max_%s"%i)
        df_label=feat_min(df_label,df_select,["air_store_id","holiday"],"visitors","air_holiday_min_%s"%i)
        df_label=feat_count(df_label,df_select,["air_store_id","holiday"],"visitors","air_holiday_count_%s"%i)

        df_label=feat_mean(df_label,df_select,["air_genre_name","holiday"],"visitors", "air_genre_name_holiday_mean_%s"%i)
        df_label=feat_max(df_label,df_select,["air_genre_name","holiday"],"visitors","air_genre_name_holiday_max_%s"%i)
        df_label=feat_min(df_label,df_select,["air_genre_name","holiday"],"visitors","air_genre_name_holiday_min_%s"%i)
        df_label=feat_count(df_label,df_select,["air_genre_name","holiday"],"visitors","air_genre_name_holiday_count_%s"%i)

        df_label=feat_mean(df_label,df_select,["air_genre_name","weekday"],"visitors", "air_genre_name_weekday_mean_%s"%i)
        df_label=feat_max(df_label,df_select,["air_genre_name","weekday"],"visitors","air_genre_name_weekday_max_%s"%i)
        df_label=feat_min(df_label,df_select,["air_genre_name","weekday"],"visitors","air_genre_name_weekday_min_%s"%i)
        df_label=feat_count(df_label,df_select,["air_genre_name","weekday"],"visitors","air_genre_name_weekday_count_%s"%i)

        df_label=feat_mean(df_label,df_select,["air_area_name","holiday"],"visitors", "air_area_name_holiday_mean_%s"%i)
        df_label=feat_max(df_label,df_select,["air_area_name","holiday"],"visitors","air_area_name_holiday_max_%s"%i)
        df_label=feat_min(df_label,df_select,["air_area_name","holiday"],"visitors","air_area_name_holiday_min_%s"%i)
        df_label=feat_count(df_label,df_select,["air_area_name","holiday"],"visitors","air_area_name_holiday_count_%s"%i)

        df_label=feat_mean(df_label,df_select,["air_area_name","air_genre_name","holiday"],"visitors", "air_area_genre_name_holiday_mean_%s"%i)
        df_label=feat_max(df_label,df_select,["air_area_name","air_genre_name","holiday"],"visitors","air_area_genre_name_holiday_max_%s"%i)
        df_label=feat_min(df_label,df_select,["air_area_name","air_genre_name","holiday"],"visitors","air_area_genre_name_holiday_min_%s"%i)
        df_label=feat_count(df_label,df_select,["air_area_name","air_genre_name","holiday"],"visitors","air_area_genre_name_holiday_count_%s"%i)
    return df_label

### 时间相关特征处理

In [19]:
# 与开始时间(t_begin)的间隔
def date_gap(x,y):
    a,b,c = x.split("-")
    return (date(int(a),int(b),int(c))-y).days

# 时间特征处理及ar hol的连接
def date_handle(df):
    df_visit_date = pd.to_datetime(df["visit_date"])
    df["weekday"] = df_visit_date.dt.weekday # 星期
    df["day"] = df_visit_date.dt.day # 天
    days_of_months = [31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]  # 一年中各个月的天数
    #  新的特征 "days_to_side" 表示当天距离月初月末的最短距离
    df["days_to_side"] = df_visit_date.apply(
        lambda x: min(x.day, days_of_months[x.month-1]-x.day))
    # 把月初月末的定义改成了7天，0 1 2
    df["day"] = df["day"].apply(lambda x:0 if x<=7 else 2 if x>=24 else 1)
    # 连接as和hol
    df = df.merge(data['as'], on="air_store_id", how="left").fillna(-1)
    #df = df.merge(hpg_info, on="hpg_store_id", how="left").fillna(-1)
    df = df.merge(data['hol'], on="visit_date", how="left").fillna(-1)
    #df["holiday"] = map(lambda a, b: 1 if a in [5, 6] or b == 1 else 0, df["weekday"], df["holiday_flg"])
    return df

## 构造训练集(扩充特征)

In [20]:
for slip in [14]:   #you can add 21 35... #,28,42
    t2017 = date(2017, 4, 23) # 2017-4-23
    nday = slip # 滑动窗口(天)

    # 构造训练集
    all_data=[]
    for i in range(nday*1,nday*(420//nday+1),nday):  # windowsize==step 滑动窗口14内天数据作为label，小于0的gap为feature
        delta = timedelta(days=i) # 时间间隔
        t_begin=t2017 - delta # 往回倒退delta天数
        print(t_begin)
        # 访问时间&预约时间与t_begin的时间差
        data['tra']["day_gap"] = data['tra']["visit_date"].apply(lambda x:date_gap(x,t_begin))
        data['ar']["day_gap"] = data['ar']["reserve_date"].apply(lambda x:date_gap(x,t_begin))
        data['hr']["day_gap"] = data['hr']["reserve_date"].apply(lambda x:date_gap(x,t_begin))
        # 筛选出时间差 < 0的数据
        df_feature = data['tra'][data['tra'].day_gap < 0].copy()
        df_air_reserve = data['ar'][data['ar'].day_gap < 0].copy()
        df_hpg_reserve = data['hr'][data['hr'].day_gap < 0].copy()
        # 筛选出时间差大于等于0天，小于14天的数据
        df_label = data['tra'][(data['tra'].day_gap >= 0)&(data['tra'].day_gap < nday)][["air_store_id","hpg_store_id","visit_date","day_gap","visitors"]].copy()
        # 创造新的特征
        train_data_tmp = create_features(df_label,df_feature,df_air_reserve,df_hpg_reserve)
        all_data.append(train_data_tmp)

    train = pd.concat(all_data)

2017-04-09
2017-03-26
2017-03-12
2017-02-26
2017-02-12
2017-01-29
2017-01-15
2017-01-01
2016-12-18
2016-12-04
2016-11-20
2016-11-06
2016-10-23
2016-10-09
2016-09-25
2016-09-11
2016-08-28
2016-08-14
2016-07-31
2016-07-17
2016-07-03
2016-06-19
2016-06-05
2016-05-22
2016-05-08
2016-04-24
2016-04-10
2016-03-27
2016-03-13
2016-02-28


## 构造测试集(扩充特征)

In [21]:
t_begin = date(2017,4, 23)
print(t_begin)
df_label = data['tes'].merge(data['id'],on="air_store_id",how="left")

df_label["day_gap"] = df_label["visit_date"].apply(lambda x:date_gap(x,t_begin))
data['tra']["day_gap"] = data['tra']["visit_date"].apply(lambda x:date_gap(x,t_begin))
data['ar']["day_gap"] = data['ar']["reserve_date"].apply(lambda x: date_gap(x, t_begin))
data['hr']["day_gap"] = data['hr']["reserve_date"].apply(lambda x: date_gap(x, t_begin))


# df_label(tes) 2017-04-23开始
df_label=df_label[["air_store_id","hpg_store_id","visit_date","day_gap"]].copy()
test = create_features(df_label,data['tra'],data['ar'],data['hr'])

2017-04-23


In [22]:
train.head()

,air_store_id,hpg_store_id,visit_date,day_gap,visitors,weekday,day,days_to_side,air_genre_name,air_area_name,latitude,longitude,holiday_flg,holiday,air_reserve_datetime_diff_sum,air_reserve_visitors_sum,air_date_reserve_visitors_sum,hpg_reserve_datetime_diff_sum,hpg_reserve_visitors_sum,hpg_date_reserve_visitors_sum,air_reserve_datetime_diff_mean,air_reserve_visitors_mean,air_date_reserve_visitors_mean,hpg_reserve_datetime_diff_mean,hpg_reserve_visitors_mean,hpg_date_reserve_visitors_mean,air_reserve_visitors_sum_weekday_mean_7,hpg_reserve_visitors_sum_weekday_mean_7,air_reserve_visitors_mean_weekday_mean_7,hpg_reserve_visitors_mean_weekday_mean_7,air_reserve_visitors_count_weekday_mean_7,hpg_reserve_visitors_count_weekday_mean_7,air_reserve_visitors_sum_holiday_mean_7,hpg_reserve_visitors_sum_holiday_mean_7,air_reserve_visitors_mean_holiday_mean_7,hpg_reserve_visitors_mean_holiday_mean_7,air_reserve_visitors_count_holiday_mean_7,hpg_reserve_visitors_count_holiday_mean_7,air_reserve_visitors_sum_weekday_mean_14,hpg_reserve_visitors_sum_weekday_mean_14,air_reserve_visitors_mean_weekday_mean_14,hpg_reserve_visitors_mean_weekday_mean_14,air_reserve_visitors_count_weekday_mean_14,hpg_reserve_visitors_count_weekday_mean_14,air_reserve_visitors_sum_holiday_mean_14,hpg_reserve_visitors_sum_holiday_mean_14,air_reserve_visitors_mean_holiday_mean_14,hpg_reserve_visitors_mean_holiday_mean_14,air_reserve_visitors_count_holiday_mean_14,hpg_reserve_visitors_count_holiday_mean_14,air_reserve_visitors_sum_weekday_mean_35,hpg_reserve_visitors_sum_weekday_mean_35,air_reserve_visitors_mean_weekday_mean_35,hpg_reserve_visitors_mean_weekday_mean_35,air_reserve_visitors_count_weekday_mean_35,hpg_reserve_visitors_count_weekday_mean_35,air_reserve_visitors_sum_holiday_mean_35,hpg_reserve_visitors_sum_holiday_mean_35,air_reserve_visitors_mean_holiday_mean_35,hpg_reserve_visitors_mean_holiday_mean_35,air_reserve_visitors_count_holiday_mean_35,hpg_reserve_visitors_count_holiday_mean_35,air_reserve_visitors_sum_weekday_mean_63,hpg_reserve_visitors_sum_weekday_mean_63,air_reserve_visitors_mean_weekday_mean_63,hpg_reserve_visitors_mean_weekday_mean_63,air_reserve_visitors_count_weekday_mean_63,hpg_reserve_visitors_count_weekday_mean_63,air_reserve_visitors_sum_holiday_mean_63,hpg_reserve_visitors_sum_holiday_mean_63,air_reserve_visitors_mean_holiday_mean_63,hpg_reserve_visitors_mean_holiday_mean_63,air_reserve_visitors_count_holiday_mean_63,hpg_reserve_visitors_count_holiday_mean_63,air_reserve_visitors_sum_weekday_mean_140,hpg_reserve_visitors_sum_weekday_mean_140,air_reserve_visitors_mean_weekday_mean_140,hpg_reserve_visitors_mean_weekday_mean_140,air_reserve_visitors_count_weekday_mean_140,hpg_reserve_visitors_count_weekday_mean_140,air_reserve_visitors_sum_holiday_mean_140,hpg_reserve_visitors_sum_holiday_mean_140,air_reserve_visitors_mean_holiday_mean_140,hpg_reserve_visitors_mean_holiday_mean_140,air_reserve_visitors_count_holiday_mean_140,hpg_reserve_visitors_count_holiday_mean_140,air_day_mean,air_holiday_mean,air_median_21,air_mean_21,air_max_21,air_min_21,air_std_21,air_count_21,air_week_mean_21,air_week_max_21,air_week_min_21,air_week_std_21,air_week_count_21,air_holiday_mean_21,air_holiday_max_21,air_holiday_min_21,air_holiday_count_21,air_genre_name_holiday_mean_21,air_genre_name_holiday_max_21,air_genre_name_holiday_min_21,air_genre_name_holiday_count_21,air_genre_name_weekday_mean_21,air_genre_name_weekday_max_21,air_genre_name_weekday_min_21,air_genre_name_weekday_count_21,air_area_name_holiday_mean_21,air_area_name_holiday_max_21,air_area_name_holiday_min_21,air_area_name_holiday_count_21,air_area_genre_name_holiday_mean_21,air_area_genre_name_holiday_max_21,air_area_genre_name_holiday_min_21,air_area_genre_name_holiday_count_21,air_median_35,air_mean_35,air_max_35,air_min_35,air_std_35,air_count_35,air_week_mean_35,air_week_max_35,air_week_min_35,air_week_std_35,air_week_count_35,air_holiday_mean_35,air_holiday_max_35,air_holiday_min_35,a

In [27]:
test.head()

,air_store_id,hpg_store_id,visit_date,day_gap,weekday,day,days_to_side,air_genre_name,air_area_name,latitude,longitude,holiday_flg,holiday,air_reserve_datetime_diff_sum,air_reserve_visitors_sum,air_date_reserve_visitors_sum,hpg_reserve_datetime_diff_sum,hpg_reserve_visitors_sum,hpg_date_reserve_visitors_sum,air_reserve_datetime_diff_mean,air_reserve_visitors_mean,air_date_reserve_visitors_mean,hpg_reserve_datetime_diff_mean,hpg_reserve_visitors_mean,hpg_date_reserve_visitors_mean,air_reserve_visitors_sum_weekday_mean_7,hpg_reserve_visitors_sum_weekday_mean_7,air_reserve_visitors_mean_weekday_mean_7,hpg_reserve_visitors_mean_weekday_mean_7,air_reserve_visitors_count_weekday_mean_7,hpg_reserve_visitors_count_weekday_mean_7,air_reserve_visitors_sum_holiday_mean_7,hpg_reserve_visitors_sum_holiday_mean_7,air_reserve_visitors_mean_holiday_mean_7,hpg_reserve_visitors_mean_holiday_mean_7,air_reserve_visitors_count_holiday_mean_7,hpg_reserve_visitors_count_holiday_mean_7,air_reserve_visitors_sum_weekday_mean_14,hpg_reserve_visitors_sum_weekday_mean_14,air_reserve_visitors_mean_weekday_mean_14,hpg_reserve_visitors_mean_weekday_mean_14,air_reserve_visitors_count_weekday_mean_14,hpg_reserve_visitors_count_weekday_mean_14,air_reserve_visitors_sum_holiday_mean_14,hpg_reserve_visitors_sum_holiday_mean_14,air_reserve_visitors_mean_holiday_mean_14,hpg_reserve_visitors_mean_holiday_mean_14,air_reserve_visitors_count_holiday_mean_14,hpg_reserve_visitors_count_holiday_mean_14,air_reserve_visitors_sum_weekday_mean_35,hpg_reserve_visitors_sum_weekday_mean_35,air_reserve_visitors_mean_weekday_mean_35,hpg_reserve_visitors_mean_weekday_mean_35,air_reserve_visitors_count_weekday_mean_35,hpg_reserve_visitors_count_weekday_mean_35,air_reserve_visitors_sum_holiday_mean_35,hpg_reserve_visitors_sum_holiday_mean_35,air_reserve_visitors_mean_holiday_mean_35,hpg_reserve_visitors_mean_holiday_mean_35,air_reserve_visitors_count_holiday_mean_35,hpg_reserve_visitors_count_holiday_mean_35,air_reserve_visitors_sum_weekday_mean_63,hpg_reserve_visitors_sum_weekday_mean_63,air_reserve_visitors_mean_weekday_mean_63,hpg_reserve_visitors_mean_weekday_mean_63,air_reserve_visitors_count_weekday_mean_63,hpg_reserve_visitors_count_weekday_mean_63,air_reserve_visitors_sum_holiday_mean_63,hpg_reserve_visitors_sum_holiday_mean_63,air_reserve_visitors_mean_holiday_mean_63,hpg_reserve_visitors_mean_holiday_mean_63,air_reserve_visitors_count_holiday_mean_63,hpg_reserve_visitors_count_holiday_mean_63,air_reserve_visitors_sum_weekday_mean_140,hpg_reserve_visitors_sum_weekday_mean_140,air_reserve_visitors_mean_weekday_mean_140,hpg_reserve_visitors_mean_weekday_mean_140,air_reserve_visitors_count_weekday_mean_140,hpg_reserve_visitors_count_weekday_mean_140,air_reserve_visitors_sum_holiday_mean_140,hpg_reserve_visitors_sum_holiday_mean_140,air_reserve_visitors_mean_holiday_mean_140,hpg_reserve_visitors_mean_holiday_mean_140,air_reserve_visitors_count_holiday_mean_140,hpg_reserve_visitors_count_holiday_mean_140,air_day_mean,air_holiday_mean,air_median_21,air_mean_21,air_max_21,air_min_21,air_std_21,air_count_21,air_week_mean_21,air_week_max_21,air_week_min_21,air_week_std_21,air_week_count_21,air_holiday_mean_21,air_holiday_max_21,air_holiday_min_21,air_holiday_count_21,air_genre_name_holiday_mean_21,air_genre_name_holiday_max_21,air_genre_name_holiday_min_21,air_genre_name_holiday_count_21,air_genre_name_weekday_mean_21,air_genre_name_weekday_max_21,air_genre_name_weekday_min_21,air_genre_name_weekday_count_21,air_area_name_holiday_mean_21,air_area_name_holiday_max_21,air_area_name_holiday_min_21,air_area_name_holiday_count_21,air_area_genre_name_holiday_mean_21,air_area_genre_name_holiday_max_21,air_area_genre_name_holiday_min_21,air_area_genre_name_holiday_count_21,air_median_35,air_mean_35,air_max_35,air_min_35,air_std_35,air_count_35,air_week_mean_35,air_week_max_35,air_week_min_35,air_week_std_35,air_week_count_35,air_holiday_mean_35,air_holiday_max_35,air_holiday_min_35,air_holida

In [23]:
# 删除无用的特征
train_data = train.drop(["air_store_id","hpg_store_id","visit_date"], axis=1)
test_data = test.drop(["air_store_id","hpg_store_id","visit_date"], axis=1)

## 训练预测

In [24]:
# 特征
col = [c for c in train_data.columns if c not in ["id", "visitors"]]

In [26]:
def RMSLE(y, pred):
    return metrics.mean_squared_error(y, pred)**0.5
    
model1 = ensemble.GradientBoostingRegressor(learning_rate=0.2, random_state=3, n_estimators=200, subsample=0.8, 
                      max_depth =10)
model2 = neighbors.KNeighborsRegressor(n_jobs=-1, n_neighbors=4)
model3 = XGBRegressor(learning_rate=0.2, random_state=3, n_estimators=280, subsample=0.8, 
                      colsample_bytree=0.8, max_depth =12)

model1.fit(train_data[col], train_data['visitors'].values)
model2.fit(train_data[col], train_data['visitors'].values)
model3.fit(train_data[col], train_data['visitors'].values)

preds1 = model1.predict(train_data[col])
preds2 = model2.predict(train_data[col])
preds3 = model3.predict(train_data[col])

print('RMSE GradientBoostingRegressor: ', RMSLE(train_data['visitors'].values, preds1))
print('RMSE KNeighborsRegressor: ', RMSLE(train_data['visitors'].values, preds2))
print('RMSE XGBRegressor: ', RMSLE(train_data['visitors'].values, preds3))
preds1 = model1.predict(test_data[col])
preds2 = model2.predict(test_data[col])
preds3 = model3.predict(test_data[col])

test_data['visitors'] = 0.3*preds1 + 0.3*preds2 + 0.4*preds3
test_data['visitors'] = np.expm1(test_data['visitors']).clip(lower=0.)
sub1 = test_data[['id','visitors']].copy()
del train; del data;

RMSE GradientBoostingRegressor:  0.3068097626973749
RMSE KNeighborsRegressor:  0.5950444145582215
RMSE XGBRegressor:  0.15589627131499434


KeyError: "['id'] not in index"